# Predicting Power Outage Severity

**Name(s)**: Matthew Jacobsen

**Website Link**: https://mfjacobsen.github.io/PowerOutagesAnalysis/

In [32]:
import pandas as pd
import numpy as np
from pathlib import Path
import openpyxl
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import plotly.express as px
pd.options.plotting.backend = 'plotly'
from dsc80_utils import * # Feel free to uncomment and use this.
from IPython.display import display
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [33]:
def display_df(df, rows=pd.options.display.max_rows, 
               cols=pd.options.display.max_columns):
    """Displays n rows and cols from df."""
    with pd.option_context("display.max_rows", rows,
                            "display.max_columns", cols): 
        display(df)

## Step 1: Introduction

This project seeks to identify the areas of the country that should be most concerned with El 
Nino and La Nina anomaly levels and how those levels will impact them

In [34]:
# Reads in the dataframe
df = pd.read_excel("outage.xlsx.xls",skiprows=5)
units = df.loc[4,:]
df = df.iloc[1:,1:].set_index("OBS")
df

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,...,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
OBS,,,,,,,,,
1.0,2011.0,7.0,Minnesota,MN,...,0.6,91.59,8.41,5.48
2.0,2014.0,5.0,Minnesota,MN,...,0.6,91.59,8.41,5.48
3.0,2010.0,10.0,Minnesota,MN,...,0.6,91.59,8.41,5.48
...,...,...,...,...,...,...,...,...,...
1532.0,2009.0,8.0,South Dakota,SD,...,0.15,98.31,1.69,1.69
1533.0,2009.0,8.0,South Dakota,SD,...,0.15,98.31,1.69,1.69
1534.0,2000.0,NaN,Alaska,AK,...,0.02,85.76,14.24,2.9


In [35]:
# Display the unique climate regions
df["CLIMATE.REGION"].unique()

array(['East North Central', 'Central', 'South', 'Southeast', 'Northwest',
       'Southwest', 'Northeast', 'West North Central', 'West', nan],
      dtype=object)

## Step 2: Data Cleaning and Exploratory Data Analysis

In [36]:
# Changes the outage start and end times to timestamps
start_time = (pd.to_datetime(df["OUTAGE.START.DATE"]) 
              + pd.to_timedelta(df["OUTAGE.START.TIME"].astype(str)))

end_time = (pd.to_datetime(df["OUTAGE.RESTORATION.DATE"]) 
              + pd.to_timedelta(df["OUTAGE.RESTORATION.TIME"].astype(str)))
df["OUTAGE.START.TIME"] = pd.to_datetime(start_time)
df["OUTAGE.RESTORATION.TIME"] = pd.to_datetime(end_time)
df = df.drop(columns=['OUTAGE.START.DATE', 'OUTAGE.RESTORATION.DATE'])

# We'll split up the outage start time into a new categorical column indicating
# the time of day the outage started

# Define time boundaries for each category
morning_start = pd.to_datetime('04:00:00').time()
morning_end = pd.to_datetime('11:59:59').time()
afternoon_start = pd.to_datetime('12:00:00').time()
afternoon_end = pd.to_datetime('19:59:59').time()
night_start = pd.to_datetime('20:00:00').time()
night_end = pd.to_datetime('03:59:59').time()

# Function to categorize timestamps
def categorize_time(timestamp):
    try:
        if morning_start <= timestamp.time() <= morning_end:
            return 'Morning'
        elif afternoon_start <= timestamp.time() <= afternoon_end:
            return 'Afternoon'
        else:
            return 'Night'
    except ValueError:
        return None

# Defines the outage start period as morning, noon, or night
df['OUTAGE.START.PERIOD'] = df['OUTAGE.START.TIME'].apply(categorize_time)

# Defines a new category called customer minutes, the total number
# of customer minutes lost for the outage
df["CUSTOMER.MINUTES"] = df["CUSTOMERS.AFFECTED"] * df["OUTAGE.DURATION"]

# The numerical data columns associated with the outage itself
NUM_COLS = ['ANOMALY.LEVEL','OUTAGE.DURATION', 'DEMAND.LOSS.MW', 
            'CUSTOMERS.AFFECTED', 'CUSTOMER.MINUTES']

# The categorical columns data columns associated with the outage and state
CAT_COLS = ['MONTH', 'U.S._STATE', 'NERC.REGION', 'CLIMATE.REGION', 
            'CLIMATE.CATEGORY', 'CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL',
            'HURRICANE.NAMES']

# The state columns contain numerical data about state for that year
STATE_COLS = ['RES.PRICE', 'COM.PRICE', 'IND.PRICE',
       'TOTAL.PRICE', 'RES.SALES', 'COM.SALES', 'IND.SALES', 'TOTAL.SALES',
       'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN', 'RES.CUSTOMERS',
       'COM.CUSTOMERS', 'IND.CUSTOMERS', 'TOTAL.CUSTOMERS', 'RES.CUST.PCT',
       'COM.CUST.PCT', 'IND.CUST.PCT', 'PC.REALGSP.STATE', 'PC.REALGSP.USA',
       'PC.REALGSP.REL', 'PC.REALGSP.CHANGE', 'UTIL.REALGSP', 'TOTAL.REALGSP',
       'UTIL.CONTRI', 'PI.UTIL.OFUSA', 'POPULATION', 'POPPCT_URBAN',
       'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC', 'POPDEN_RURAL',
       'AREAPCT_URBAN', 'AREAPCT_UC', 'PCT_LAND', 'PCT_WATER_TOT',
       'PCT_WATER_INLAND']

# Columns that describe the time of the outage
TIME_COLS = ['YEAR', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.TIME', 
             'OUTAGE.START.PERIOD']

# Note: CLIMATE.CATEGORY is a grouping of ANOMALY.LEVEL:
#       cool: level < -0.5 ; normal: -0.5 < level < 0.5 ; warm: 0.5 < level

rel_cols = ['MONTH','CLIMATE.REGION', 'CAUSE.CATEGORY.DETAIL',
             'OUTAGE.START.PERIOD', 'ANOMALY.LEVEL', 'CUSTOMERS.AFFECTED',

             'OUTAGE.DURATION', "DEMAND.LOSS.MW"]

df_rel = df[rel_cols]
print(df_rel.head().to_markdown(index=True))

|   OBS |   MONTH | CLIMATE.REGION     | CAUSE.CATEGORY.DETAIL   | OUTAGE.START.PERIOD   |   ANOMALY.LEVEL |   CUSTOMERS.AFFECTED |   OUTAGE.DURATION |   DEMAND.LOSS.MW |
|------:|--------:|:-------------------|:------------------------|:----------------------|----------------:|---------------------:|------------------:|-----------------:|
|     1 |       7 | East North Central | nan                     | Afternoon             |            -0.3 |                70000 |              3060 |              nan |
|     2 |       5 | East North Central | vandalism               | Afternoon             |            -0.1 |                  nan |                 1 |              nan |
|     3 |      10 | East North Central | heavy wind              | Night                 |            -1.5 |                70000 |              3000 |              nan |
|     4 |       6 | East North Central | thunderstorm            | Morning               |            -0.1 |                68200 |              

In [37]:
# Plots a histogram that shows the distribution of anomaly levels.
fig = px.histogram(df, x = "ANOMALY.LEVEL", nbins = 20)
fig.update_layout(title = 'Distribution of Anomaly Level',
                  yaxis_title = "Number of Outages",
                  xaxis_title = "Anomaly Level")
fig.show()

In [38]:
# Plots a histogram showing the distribution of outage durations
fig = px.histogram(df, x = "OUTAGE.DURATION")
fig.update_layout(title = 'Distribution of Outage Durations',
                  yaxis_title = "Number of Outages",
                  xaxis_title = "Outage Duration",
                  xaxis = dict(range=[0, 25000]))
fig.show()
fig.write_html('Outage Duration Distribution.html', include_plotlyjs='cdn')

In [39]:
# Plots a histogram showing the distribution of customers affected
fig = px.histogram(df, x = "CUSTOMERS.AFFECTED")
fig.update_layout(title = 'Distribution of Customers Affected',
                  yaxis_title = "Number of Outages",
                  xaxis_title = "Customers Affected")
fig.show()

In [40]:
# Plots a scatter plot showing the relationship of number of customers affected
# and anomaly level
fig = px.scatter(df, x = "ANOMALY.LEVEL", y = "CUSTOMERS.AFFECTED")
fig.update_layout(title = 'Customers Affected vs Anomaly Level',
                  xaxis_title = "Anomaly Level",
                  yaxis_title = "Customers Affected")

In [41]:
# Plots a scatter plot showing the relationship of number of customers affected
# and outage duration
fig = px.scatter(df, x = "ANOMALY.LEVEL", y = "OUTAGE.DURATION",range_y=[0, 35000])
fig.update_layout(title = 'Outage Duration vs Anomaly Level',
                  xaxis_title = "Anomaly Level",
                  yaxis_title = "Outage Duration")
fig.write_html('Duration vs Level.html', include_plotlyjs='cdn')


In [42]:
# This pivot table shows the average outage duration for each climate region
# for each climate categoory. We can see that some are quite different
piv = df.pivot_table(index="CLIMATE.REGION", columns = 'CLIMATE.CATEGORY', 
                     values = 'OUTAGE.DURATION', aggfunc = 'mean')
print(piv.to_markdown(index=True))

| CLIMATE.REGION     |     cold |    normal |    warm |
|:-------------------|---------:|----------:|--------:|
| Central            | 2799.86  | 2708.7    | 2413.84 |
| East North Central | 6568.79  | 5271.22   | 3022.12 |
| Northeast          | 3657.25  | 2261.33   | 4175.91 |
| Northwest          |  874.681 |  733.612  | 3063.54 |
| South              | 2012.71  | 3753.06   | 1861.4  |
| Southeast          | 1707.07  | 2392.27   | 2528.94 |
| Southwest          |  544.591 |  296.136  | 5127.68 |
| West               | 1762.71  | 1249.84   | 2044.23 |
| West North Central |  200     |   28.4286 | 2486.5  |


## Step 3: Assessment of Missingness

The OUTAGE.DURATION column is likely NMAR. It's probable that outages whose 
duration were less than a few seconds did not have their duration recorded 
during reporting. It may be that reporters did not understand the value of 
having an outage of zero minutes in duration. Additional information that could
have been collected to make this NMAR would be to include a column that 
categorizes the outage as an 'outage' or a 'hit', with a power hit being anything
and outage with a a duration less than a few seconds.

Now we'll test if CAUSE.CATEGORY.DETAIL is MAR with respect to state using 
permutation testing with total variation distance as our test statistic. The 
null hypothesis is that there is no difference.

In [43]:
# Determines the p value of the missingness of CAUSE.CATEGORY.DETAIL with 
# respect to the passed column
def get_detail_miss(col):
    miss_df = df[[col]]
    miss_df["MISSING.DETAIL"] = df["CAUSE.CATEGORY.DETAIL"].isna()

    def get_TVD(miss_df):
        miss_dist = (miss_df[miss_df["MISSING.DETAIL"]][col].value_counts()
                / miss_df[miss_df["MISSING.DETAIL"]].shape[0]) 
        n_miss_dist = (miss_df[~miss_df["MISSING.DETAIL"]][col].value_counts()
                / miss_df[~miss_df["MISSING.DETAIL"]].shape[0]) 
        test_stat = abs(miss_dist - n_miss_dist).sum() / 2
        return test_stat

    test_stat = get_TVD(miss_df)

    test_stats = []
    B = 1000
    for i in range(B):
        miss_df["MISSING.DETAIL"] = (
            np.random.permutation(miss_df["MISSING.DETAIL"]))
        test_stats.append(get_TVD(miss_df))

    p_value = ((test_stat <= np.array(test_stats)).sum() + 1) / (B + 1)
    return p_value

In [44]:
# Let's look at the missingness mechanism for the following columns
columns = ['MONTH', 'U.S._STATE', 'NERC.REGION', 'CLIMATE.REGION', 
            'CLIMATE.CATEGORY', 'CAUSE.CATEGORY', 'OUTAGE.START.PERIOD']

p_values = pd.Series({col:get_detail_miss(col) for col in columns},
                     name = "P Value").sort_values()

# We can see that CAUSE.CATEGORY.DETAIL is missing at random with respect to 
# Month, State, NERC Region, Climate Region, and Climate Category, but not with
# respect to Outage Start Period (morning, afternoon, or night).
p_values

MONTH                  9.99e-04
U.S._STATE             9.99e-04
NERC.REGION            9.99e-04
CLIMATE.REGION         9.99e-04
CAUSE.CATEGORY         9.99e-04
CLIMATE.CATEGORY       2.00e-03
OUTAGE.START.PERIOD    3.03e-01
Name: P Value, dtype: float64

In [45]:
pd.DataFrame(p_values)
print((pd.DataFrame(p_values)).to_markdown(index=True))

|                     |     P Value |
|:--------------------|------------:|
| MONTH               | 0.000999001 |
| U.S._STATE          | 0.000999001 |
| NERC.REGION         | 0.000999001 |
| CLIMATE.REGION      | 0.000999001 |
| CAUSE.CATEGORY      | 0.000999001 |
| CLIMATE.CATEGORY    | 0.001998    |
| OUTAGE.START.PERIOD | 0.302697    |


## Step 4: Hypothesis Testing

In [46]:
# Lets determine if the average outage duration is the same for each  
# CLIMATE.CATEGORY, the climate categories corespond to ANOMOLY.LEVELS where:
# cool: level < -0.5 ; normal: -0.5 < level < 0.5 ; warm: 0.5 < level
# First we can look at the mean outage duration by anomaly level

def test_duration(region = "all"):

    if region == "all":
        mean_duration = df["OUTAGE.DURATION"].mean()
        category_df = df[["CLIMATE.CATEGORY", "OUTAGE.DURATION"]]
    
    else:
        mean_duration = (df[df["CLIMATE.REGION"] == region]
                         ["OUTAGE.DURATION"].mean())
        category_df = (df[df["CLIMATE.REGION"] == region]
                       [["CLIMATE.CATEGORY", "OUTAGE.DURATION"]])

    def calcualte_MSE(df):
        mean_durations = (df.groupby("CLIMATE.CATEGORY")
                          ["OUTAGE.DURATION"].mean())
        return ((mean_durations - mean_duration) ** 2).mean()

    test_stat = calcualte_MSE(category_df)

    test_stats = []
    B = 1000
    for i in range(B):
        category_df["CLIMATE.CATEGORY"] = (
            np.random.permutation(category_df["CLIMATE.CATEGORY"]))
        test_stats.append(calcualte_MSE(category_df))

    p_value = ((test_stat <= np.array(test_stats)).sum() + 1) / (B + 1)
    return (p_value, test_stats)

# We can see no difference in average outage duration, but what if we test for
# each climate region individually?
p_value, test_stats = test_duration()


In [47]:
# Get's the p values for each region
region_durations = pd.Series({region:test_duration(region)[0] for region in 
                              df["CLIMATE.REGION"].unique()}, name='P Value')


In [48]:
# With multiple hypothesis testing like this, it's good practice to control for 
# the increased risk of a type one error rate by adjusting the p values. We will
# do so with the Bonferroni method. With ten hypothesis tests, we will reject the
# null at a significance level of 0.05 / 10 = 0.005 or 5.00e-3. Therefore, we 
# reject the null hypothesis for the Northwest region, it appears states in the 
# Northwest region do nat have the same average outage duration for each 
# Climate Region
region_durations_df = pd.DataFrame(region_durations.sort_values())
print(region_durations_df.to_markdown(index=True))

|                    |     P Value |
|:-------------------|------------:|
| Northwest          | 0.000999001 |
| nan                | 0.000999001 |
| Southwest          | 0.030969    |
| South              | 0.046953    |
| Northeast          | 0.048951    |
| West North Central | 0.118881    |
| East North Central | 0.506494    |
| Southeast          | 0.55045     |
| West               | 0.667333    |
| Central            | 0.888112    |


## Step 5: Framing a Prediction Problem

In [49]:
# Let's try and predict the outage duration outages using the month, climate category,
# and anomaly level. It's reasonable to guess outage duration varies by month, 
# and if anomaly level has an impact on sever weather, it may in turn impact the 
# duration of outages.

## Step 6: Baseline Model

In [50]:
# Calculates the root mean squared loss of two vectors
def calc_rmse(y_predict, y_actual):
    return np.sqrt(((y_predict - y_actual)**2).mean())

In [51]:
# Relevant feature column names for our anlaysis
all_cols = ['MONTH','CLIMATE.REGION', 'CAUSE.CATEGORY.DETAIL',
             'OUTAGE.START.PERIOD', 'ANOMALY.LEVEL', 'CUSTOMERS.AFFECTED',
             'OUTAGE.DURATION', "DEMAND.LOSS.MW"]
cat_cols = ['MONTH','CLIMATE.REGION', 'CAUSE.CATEGORY.DETAIL',
             'OUTAGE.START.PERIOD']
num_cols = ['ANOMALY.LEVEL', 'CUSTOMERS.AFFECTED']

# The column name we want to predicty
response_col = 'OUTAGE.DURATION'

# Creates the dataframe for our model
df_mod = df[all_cols].dropna()


In [52]:
# Our basline model will try and predict outage duration from anomaly level, month, and climate region
seed = 2

# The categorical and numerical columns for the model
base_cat_cols = ['MONTH', 'CLIMATE.REGION']
base_num_cols = ['ANOMALY.LEVEL']
base_drop_cols = [col for col in cat_cols + num_cols 
                  if col not in base_num_cols + base_cat_cols]

# Split the features dataframe and response series
features = df_mod[cat_cols + num_cols]
response = df_mod[response_col]

# Split into training and testing sets
X_train, X_test, y_train, y_test = (
    train_test_split(features, response,random_state=seed))

# Define the column transformer and pipeline
processing_base = ColumnTransformer(
        transformers = [('cat_cols', OneHotEncoder(handle_unknown='ignore',
                                                drop='first'), base_cat_cols),
                        ('numeric', 'passthrough', base_num_cols),
                        ('drop', 'drop', base_drop_cols)]
    )
pl_base = Pipeline([
    ('preprocessor', processing_base),
    ('regressor', RandomForestRegressor(random_state=seed))
])

# Fit and evaluate the model
pl_base.fit(X_train, y_train)
predicted = pl_base.predict(X_test)

rmse_base = calc_rmse(predicted, y_test)
score_base = pl_base.score(X_test, y_test)

print(f'Base Model r sq: {score_base}')
print(f'Base Model RMSE: {rmse_base}')

Base Model r sq: -0.07798395199365982
Base Model RMSE: 3741.535267641852


## Step 7: Final Model

In [53]:
# Let's refine the model we'll start by adding in some more variables:
# Climate Region, Cause category Detail, Outage Start Period (morning, noon, night),
# and Customers Affected. We'll binarizer the customers affected column
# and we'll transform the anomaly level to the absolute value of the level. This
# will model the effects of anomaly level as a deviation from normal in either direction
processing_final = ColumnTransformer(
        transformers = [('cat_cols', 
                         OneHotEncoder(handle_unknown='ignore', 
                                       drop='first'), cat_cols),
                        ('binarize', 
                         Binarizer(threshold=200000), 
                         ['CUSTOMERS.AFFECTED']),
                        ('anomaly level', 
                         FunctionTransformer(lambda x: np.abs(x)), 
                         ['ANOMALY.LEVEL'])], 
        remainder = 'passthrough'
    )
pl_final = Pipeline([
    ('preprocessor', processing_final),
    ('regressor', RandomForestRegressor(random_state=seed))
])

# Next we'll do a grid search to find the optimal hyperparameters of our Decision Tree Regressor
hyperparameters = {
    'regressor__max_depth' : [4, 5, 6, 7, 8, 9, 10],
    'regressor__max_features': [3, 4, 5],
    'regressor__min_samples_split': [3, 4, 5, 6]
}

# Performs the gridsearch
grids = GridSearchCV(pl_final, hyperparameters, cv=10, n_jobs=-1)
grids.fit(X_train, y_train)

#Extracts the hyperparameters
params = grids.best_params_
params = {param[11:]: value for (param, value) in params.items()}
params

/opt/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:170: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:170: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:170: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:170: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/anaconda3/envs/dsc80/lib/python3.8/site-packages/sk

{'max_depth': 8, 'max_features': 4, 'min_samples_split': 4}

In [54]:
# We'll use the best hyperparameters from above in our final model pipeline
pl_final = Pipeline([
    ('preprocessor', processing_final),
    ('regressor', RandomForestRegressor(**params,random_state=seed))
])

# Fit and evaluate the model
pl_final.fit(X_train, y_train)
predicted = pl_final.predict(X_test)

rmse_final = calc_rmse(predicted, y_test)
score_final = pl_final.score(X_test, y_test)

print(f'Base Model r sq: {score_final}')
print(f'Base Model RMSE: {rmse_final}')

Base Model r sq: 0.4054063847845649
Base Model RMSE: 2778.7777717555437


In [56]:
model_eval = pd.DataFrame([{'RMSE': rmse_base, 'r Squared': score_base},
              {'RMSE': rmse_final, 'r Squared': score_final}],
              index = ['Base', 'Final'])
print(model_eval.to_markdown(index=True))

|       |    RMSE |   r Squared |
|:------|--------:|------------:|
| Base  | 3741.54 |   -0.077984 |
| Final | 2778.78 |    0.405406 |


## Step 8: Fairness Analysis

In [25]:
# Lets look at how fair our model is for each climate region with a hypothesis test
df_fair = features.copy()
df_fair["OUTAGE.DURATION.PREDICTED"] = pl_final.predict(features)
df_fair["OUTAGE.DURATION"] = response.astype(float)
df_fair = df_fair[["CLIMATE.REGION", 
                   "OUTAGE.DURATION.PREDICTED", "OUTAGE.DURATION"]]

# Calculate the root mean squared error for each climate region
rmses = {region: calc_rmse(df_fair[df_fair["CLIMATE.REGION"] == region]
                           ["OUTAGE.DURATION.PREDICTED"],
                           df_fair[df_fair["CLIMATE.REGION"] == region]
                           ["OUTAGE.DURATION"])
         for region in df_fair["CLIMATE.REGION"].unique()}

# We can see that there is quite a difference in each region, we'll proceed with a 
# permutation test
fair_rmse = pd.DataFrame(pd.Series(rmses, name='RMSE').sort_values())
print(fair_rmse.to_markdown(index=True))


|                    |     RMSE |
|:-------------------|---------:|
| Southwest          |  838.442 |
| Northwest          | 1899.61  |
| Northeast          | 1981.06  |
| West North Central | 2120.92  |
| West               | 2760.61  |
| Southeast          | 3030.48  |
| Central            | 3804.56  |
| South              | 4680.82  |
| East North Central | 5157.54  |


In [26]:
# Computes the test statistic for our test, root mean squared error. To be clear,
# We are using the root mean squared error on the rmse's as our test stat.
def compute_test_stat(df_fair):
    rmses = pd.Series({region: calc_rmse(df_fair[df_fair["CLIMATE.REGION"] 
                                == region]["OUTAGE.DURATION.PREDICTED"],
            df_fair[df_fair["CLIMATE.REGION"] == region]["OUTAGE.DURATION"])
            for region in df_fair["CLIMATE.REGION"].unique()})
    return np.sqrt(((rmses - rmses.mean())**2).mean())

In [31]:
# Null Hypotheses: Our model is fair. The root mean squared error is approximately equal
# for each region.

# Alternative Hypothesis. Our model is not fair. The root mean squared error is not 
# equal in at least one rgion.

# Computes the observed test statistic
test_stat = compute_test_stat(df_fair)

# Permutes the climate region column, the recomputes the root mean squared error
# for each region. The takes the root mean squared error of the regional root mean 
# squared errors. Under the null hypothesis, we expect the test statistic to be 
# low. We'll reject the null for high values of the the test statistc. 
test_stats = []
B = 10000
for i in range(B):
    df_fair["CLIMATE.REGION"] = (
        np.random.permutation(df_fair["CLIMATE.REGION"]))
    test_stats.append(compute_test_stat(df_fair))

#Computes the p value
p_value = (np.array(test_stats) >= test_stat).sum() / B

# It seems at the 0.05 significance level, that our model is fair 
# across each climate region
p_value

0.1654